In [1]:
!pip install flask flask_sqlalchemy flask_bcrypt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 6.4 MB/s eta 0:00:00


In [6]:
from flask import Flask, request, jsonify
from flask_sqlalchemy import SQLAlchemy
from flask_bcrypt import Bcrypt

app = Flask(__name__)

app.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:///users.db'
app.config['SQLALCHEMY_TRACK_MODIFICATIONS'] = False

db = SQLAlchemy(app)
bcrypt = Bcrypt(app)

class User(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    email = db.Column(db.String(50), unique=True, nullable=False)
    password = db.Column(db.String(200), nullable=False)

with app.app_context():
    db.create_all()

@app.route('/register', methods=['POST'])
def register():
    data = request.get_json()
    hashed_password = bcrypt.generate_password_hash(data['password']).decode('utf-8')
    user = User(email=data['email'], password=hashed_password)
    db.session.add(user)
    db.session.commit()
    return jsonify({"message": "User registered securely"})

@app.route('/login', methods=['POST'])
def login():
    data = request.get_json()
    user = User.query.filter_by(email=data['email']).first()

    if user and bcrypt.check_password_hash(user.password, data['password']):
        return jsonify({"message": "Login successful"})
    else:
        return jsonify({"message": "Invalid credentials"}), 401

print("Secure Login System Ready")

Secure Login System Ready


In [3]:
# TESTING REGISTER

with app.app_context():
    test_email = "test@gmail.com"
    test_password = "mypassword123"

    hashed = bcrypt.generate_password_hash(test_password).decode('utf-8')

    test_user = User(email=test_email, password=hashed)
    db.session.add(test_user)
    db.session.commit()

    print("User Registered")
    print("Email:", test_email)
    print("Stored Password (Hashed):", hashed)

User Registered
Email: test@gmail.com
Stored Password (Hashed): $2b$12$xTAsHi5HBqCyHTgnc1I6Y.CieCBNHe8LefJKxqQrzL7Jg/8EJpdB2


In [7]:
# TESTING LOGIN (CORRECT PASSWORD)

with app.app_context():
    user = User.query.filter_by(email="test@gmail.com").first()

    if bcrypt.check_password_hash(user.password, "mypassword123"):
        print("Login Successful")
    else:
        print("Login Failed")

Login Successful


In [8]:
# TESTING LOGIN (WRONG PASSWORD)

with app.app_context():
    user = User.query.filter_by(email="test@gmail.com").first()

    if bcrypt.check_password_hash(user.password, "wrongpass"):
        print("Login Successful")
    else:
        print("Invalid Credentials")

Invalid Credentials
